# Requests et BeautifulSoup

## Installation Des Packages

In [1]:
# Si nous n'avez pas encore installé ces packages il faut enlever les les lignes suivantes en commentaire etles executer
# !pip install requests --upgrade --quiet
# !pip install beautifulsoup4 --upgrade --quiet
# !pip install pandas --quiet

## Importation des Packages

In [2]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

## Comment sa fonctionne

La première étape consiste à utiliser le package <code>**Requests**</code> pour télécharger le code HTML de notre page Web (à l'aide de la fonction <code>**requests.get**</code>) qui donne un objet de réponse. Nous pouvons voir si nous pouvons télécharger la page Web ou non en vérifiant la réponse. Statut de notre objet. (Il devrait être compris entre <code>**200 et 299**</code>). Ensuite, nous allons convertir cet objet de réponse en un objet <code>**BeautifulSoup**</code> en utilisant le constructeur <code>**BeautifulSoup()**</code>. Nous utiliserons cet objet pour inspecter notre document et extraire les données souhaitées. <code>**BeautifulSoup**</code> nécessite un argument supplémentaire appelé <code>**parser**</code>. (En bref, **BeautifulSoup** peut également être utilisé pour extraire des informations d'autres langages de balisage. Le **parser** par défaut est <code>**"html.parser"**</code>). Trouvez ci-dessous une fonction d'exemple indépendante qui effectuera la tâche ci-dessus de téléchargement des pages, de vérification des réponses et d'analyse à l'aide d'une belle soupe.